# Εργαστηριακή Άσκηση 2. Μη επιβλεπόμενη μάθηση. 
## Σύστημα συστάσεων βασισμένο στο περιεχόμενο
## Σημασιολογική απεικόνιση δεδομένων με χρήση SOM 
Ημερομηνία εκφώνησης της άσκησης: 29 Νοεμβρίου 2022

**Θα βρείτε το παρόν σε μορφή jupyter notebook ως συνημμένο στο τέλος της εκφώνησης.**


Neural Networks - Team 22\
\
Alexandros Tsafos el18211\
Christina Kostaki el18136\
Sofia Vouzika el18843

In [76]:
!pip install --upgrade pip
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade nltk
!pip install --upgrade scikit-learn
!pip install --upgrade joblib
!pip install --upgrade threadpoolctl

## Εισαγωγή του Dataset

Το σύνολο δεδομένων με το οποίο θα δουλέψουμε είναι βασισμένο στο [Carnegie Mellon Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/). Πρόκειται για ένα dataset με 22.301 περιγραφές ταινιών. Η περιγραφή κάθε ταινίας αποτελείται από τον τίτλο της, μια ή περισσότερες ετικέτες που χαρακτηρίζουν το είδος της ταινίας και τέλος τη σύνοψη της υπόθεσής της. Αρχικά εισάγουμε το dataset (χρησιμοποιήστε αυτούσιο τον κώδικα, δεν χρειάζεστε το αρχείο csv) στο dataframe `df_data_1`: 

In [22]:
import pandas as pd

dataset_url = "https://drive.google.com/uc?export=download&id=1zo13kUAf-MDMPZmBDxq1FxWtZY01lsxD"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

Κάθε ομάδα θα δουλέψει σε **ένα μοναδικό υποσύνολο 5.000 ταινιών** (διαφορετικό dataset για κάθε ομάδα) ως εξής:

1. Κάθε ομάδα του εργαστηρίου νευρωνικών έχει έναν αριθμό στο helios. Θα βάλετε τον αριθμό αυτό στη μεταβλητή team_seed_number στο επόμενο κελί κώδικα.

2. Το data frame `df_data_2` έχει γραμμές όσες και οι ομάδες και 5.000 στήλες. Σε κάθε ομάδα αντιστοιχεί η γραμμή του πίνακα με το `team_seed_number` της. Η γραμμή αυτή θα περιλαμβάνει 5.000 διαφορετικούς αριθμούς που αντιστοιχούν σε ταινίες του αρχικού dataset. 

3. Τρέξτε τον κώδικα. Θα προκύψουν τα μοναδικά για κάθε ομάδα  titles, categories, catbins, summaries και corpus με τα οποία θα δουλέψετε.

In [23]:
import numpy as np

# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 22

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1g6F4TCHrs2wgtdOk7D3gtONaeirNt_Vo"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

# επιλέγεται 
my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index] # movie titles (string)
categories = df_data_1.iloc[:, [3]].values[my_index] # movie categories (string)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index] # movie categories in binary form (1 feature per category)
summaries =  df_data_1.iloc[:, [5]].values[my_index] # movie summaries (string)
corpus = summaries[:,0].tolist() # list form of summaries
corpus_df = pd.DataFrame(corpus) # dataframe version of corpus

titles = list(map(lambda x : x[0], titles))
categories = list(map(lambda x : x[0], categories))

- Ο πίνακας **titles** περιέχει τους τίτλους των ταινιών. Παράδειγμα: 'Sid and Nancy'.
- O πίνακας **categories** περιέχει τις κατηγορίες (είδη) της ταινίας υπό τη μορφή string. Παράδειγμα: '"Tragedy",  "Indie",  "Punk rock",  "Addiction Drama",  "Cult",  "Musical",  "Drama",  "Biopic \[feature\]",  "Romantic drama",  "Romance Film",  "Biographical film"'. Παρατηρούμε ότι είναι μια comma separated λίστα strings, με κάθε string να είναι μια κατηγορία.
- Ο πίνακας **catbins** περιλαμβάνει πάλι τις κατηγορίες των ταινιών αλλά σε δυαδική μορφή ([one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)). Έχει διαστάσεις 5.000 x 322 (όσες οι διαφορετικές κατηγορίες). Αν η ταινία ανήκει στο συγκεκριμένο είδος η αντίστοιχη στήλη παίρνει την τιμή 1, αλλιώς παίρνει την τιμή 0.
- Ο πίνακας **summaries** και η λίστα **corpus** περιλαμβάνουν τις συνόψεις των ταινιών (η corpus είναι απλά ο summaries σε μορφή λίστας). Κάθε σύνοψη είναι ένα (συνήθως μεγάλο) string. Παράδειγμα: *'The film is based on the real story of a Soviet Internal Troops soldier who killed his entire unit  as a result of Dedovschina. The plot unfolds mostly on board of the prisoner transport rail car guarded by a unit of paramilitary conscripts.'*
- το dataframe **corpus_df** που είναι απλά το corpus σε μορφή dataframe. Τα summaries βρίσκονται στην κολόνα 0. Πιθανώς να σας βολεύει να κάνετε κάποιες προεπεξεργασίες με dataframes.


Θεωρούμε ως **ID** της κάθε ταινίας τον αριθμό γραμμής της ή το αντίστοιχο στοιχείο της λίστας. Παράδειγμα: για να τυπώσουμε τη σύνοψη της ταινίας με `ID=999` (την χιλιοστή) θα γράψουμε `print(corpus[999])`.

In [24]:
ID = 22
print(len(titles))
print(titles[ID])
print(categories[ID])
print(catbins[ID])
print(corpus[ID])

5000
Tanga
"Comedy"
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

# Εφαρμογή 1. Υλοποίηση συστήματος συστάσεων ταινιών βασισμένο στο περιεχόμενο
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

Η πρώτη εφαρμογή που θα αναπτύξετε θα είναι ένα [σύστημα συστάσεων](https://en.wikipedia.org/wiki/Recommender_system) ταινιών βασισμένο στο περιεχόμενο (content based recommender system). Τα συστήματα συστάσεων στοχεύουν στο να προτείνουν αυτόματα στο χρήστη αντικείμενα από μια συλλογή τα οποία ιδανικά θέλουμε να βρει ενδιαφέροντα ο χρήστης. Η κατηγοριοποίηση των συστημάτων συστάσεων βασίζεται στο πώς γίνεται η επιλογή (filtering) των συστηνόμενων αντικειμένων. Οι δύο κύριες κατηγορίες είναι η συνεργατική διήθηση (collaborative filtering) όπου το σύστημα προτείνει στο χρήστη αντικείμενα που έχουν αξιολογηθεί θετικά από χρήστες που έχουν παρόμοιο με αυτόν ιστορικό αξιολογήσεων και η διήθηση με βάση το περιεχόμενο (content based filtering), όπου προτείνονται στο χρήστη αντικείμενα με παρόμοιο περιεχόμενο (με βάση κάποια χαρακτηριστικά) με αυτά που έχει προηγουμένως αξιολογήσει θετικά.

Το σύστημα συστάσεων που θα αναπτύξετε θα βασίζεται στο **περιεχόμενο** και συγκεκριμένα στις συνόψεις των ταινιών (corpus). 


## Προεπεξεργασία

Το πρώτο βήμα στην επεξεργασία μας είναι ο καθαρισμός των περιγραφών των ταινιών. 

Εκτυπώστε (αρκετές) διαφορετικές περιγραφές ταινιών για να δείτε πιθανά προβλήματα που θα πρέπει να αντιμετωπιστούν.

Τα (ελάχιστα) βήματα καθαρισμού που προτείνουμε είναι:
- μετατροπή όλων των χαρακτήρων σε πεζά,
- αφαίρεση των stopwords. Εδώ σημειώστε ότι για το δεδομένο task του συστήματος συστάσεων που είναι η πρόταση ταινιών ίσως θα είχαν ενδιαφέρον και λίστες stopwords πέραν αυτών της κοινής γλώσσας.
- αφαίρεση σημείων στίξης και ειδικών χαρακτρήρων (special characters). Αυτό δεν γίνεται μόνο με την punkt του NLTK. Θα μπορούσατε να βασιστείτε σε κανονικές εκφράσεις (regular expressions), και
- αφαίρεση πολυ σύντομων συμβολοσειρών.

Προσοχή: το corpus και τα τελικά tokens που θα το αποτελούν θα χρησιμοποιηθούν στη συνέχεια ως κλειδιά για να βρούμε εμφυτεύματα. Για το λόγο αυτό, πρέπει να είστε προσεκτικοί ως προς την εφαρμογή μεθόδων κανονικοποίησης (text normalization) όπως το stemming και το lemmatization.

### Μετατροπή όλων των χαρακτήρων σε πεζούς

In [25]:
# Convert the strings to lowercase using the map() function and the lower() method
lowercase_corpus = list(map(lambda x: x.lower(), corpus))

# Print an example
print(lowercase_corpus[200])

 the first part is a bitter, tragicomic story of dzidziuś, a street-wise bon-vivant, drunkard, and coward who unwillingly becomes a soldier in the home army during the warsaw uprising. dzidziuś wife zosia is having an affair with a hungarian officer stationed nearby, and dzidziuś is ordered to contact the hungarian unit and convince the officer to join the battle against the nazis.  the second novel is set in a pow camp for polish soldiers. lt. zawistowski, one of the interned soldiers, decides to make an attempt to escape from the camp. while none of his fellow inmates are sure whether he succeeded, his absence upsets the guards and provides hope and inspiration for the rest of prisoners. soon his legend grows, making him a hero within the camp and helping to boost the prisoners' morale. however, it turns out that lt. zawistowski didn't actually follow through on his escape plans, but is hiding in the attic of one of the barracks. it turns out that he was hiding from his colleagues, w

### Αφαίρεση Σημείων Στίξης

In [26]:
import re

# Define a function that takes the corpus array and removes punctuation
def remove_punctuation(description):
    # Use a regular expression to remove punctuation
    return re.sub(r'[^\w\s]', '', description)

# Apply the function to each string in the array using the map() function
punct_corpus = list(map(remove_punctuation, lowercase_corpus))

# Print an example
print(punct_corpus[200])

 the first part is a bitter tragicomic story of dzidziuś a streetwise bonvivant drunkard and coward who unwillingly becomes a soldier in the home army during the warsaw uprising dzidziuś wife zosia is having an affair with a hungarian officer stationed nearby and dzidziuś is ordered to contact the hungarian unit and convince the officer to join the battle against the nazis  the second novel is set in a pow camp for polish soldiers lt zawistowski one of the interned soldiers decides to make an attempt to escape from the camp while none of his fellow inmates are sure whether he succeeded his absence upsets the guards and provides hope and inspiration for the rest of prisoners soon his legend grows making him a hero within the camp and helping to boost the prisoners morale however it turns out that lt zawistowski didnt actually follow through on his escape plans but is hiding in the attic of one of the barracks it turns out that he was hiding from his colleagues whose ostentatious patriot

### Αφαίρεση Stopwords και Stemming

In [27]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
  

# stopwords from NLTK
my_stopwords = stopwords.words('english')
# my new custom stopwords
my_extra = ['didnt', 'within', 'none', 'however','make', 
           'makes', 'like','well','also', 'get', 'gets', 'take',
          'takes', 'due', 'later', 'isnt', 'cant', 'dont', 'doesnt',
           'wasnt', 'werent', 'arent', 'shouldnt', 'couldnt', 'wouldnt', 
           'would', 'should', 'wants', 'seen', 'much', 'more', 'he', 'she']
# add the new custom stopwrds to my stopwords
my_stopwords.extend(my_extra)

def remove_stopwords(desc):
    text_tokens = desc.split()
    words = [word for word in text_tokens if (word not in my_stopwords) and (len(word) > 2)]
    return ' '.join(words)

def stem_desc(desc):
    ps = PorterStemmer()
    text_tokens = desc.split()
    words = [ps.stem(word) for word in text_tokens]
    return ' '.join(words)

pre_stem_corpus = list(map(remove_stopwords, punct_corpus))
final_corpus = list(map(stem_desc, pre_stem_corpus))
#print(final_corpus[0000])

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Μετατροπή σε TFIDF

Το πρώτο βήμα θα είναι λοιπόν να μετατρέψετε το corpus σε αναπαράσταση tf-idf:

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create sparse tf_idf representation
vectorizer = TfidfVectorizer(use_idf=True)
tf_idf_matrix = vectorizer.fit_transform(final_corpus)
print(tf_idf_matrix.shape)


(5000, 39266)


Η συνάρτηση [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) όπως καλείται εδώ **δεν είναι βελτιστοποιημένη**. Οι επιλογές των μεθόδων και παραμέτρων της μπορεί να έχουν **δραματική επίδραση στην ποιότητα των συστάσεων** και είναι διαφορετικές για κάθε dataset. Επίσης, οι επιλογές αυτές έχουν πολύ μεγάλη επίδραση και στη **διαστατικότητα και όγκο των δεδομένων**. Η διαστατικότητα των δεδομένων με τη σειρά της θα έχει πολύ μεγάλη επίδραση στους **χρόνους εκπαίδευσης**, ιδιαίτερα στη δεύτερη εφαρμογή της άσκησης.

Προσοχή: ο TfidfVectorizer έχει κάποιες δυνατότητες προεπεξεργασίας παρόποιες με αυτές που αναφέραμε στην προηγούμενη ενότητα. Ό,τι προεπεξεργασία μπορείτε να κάνετε που χρειάζεται ως είσοδο μόνο το κάθε document ξεχωριστά, κάντε την στο πρώτο βήμα της προεπεξεργασίας. Αν χρειάζεται γνώση των συνολικών στατιστικών της συλλογής, κάντε την με τον TfidfVectorizer.

## Υλοποίηση του συστήματος συστάσεων

Το σύστημα συστάσεων που θα υλοποιήσετε θα είναι μια συνάρτηση `content_recommender` με τρία ορίσματα: `target_movie`, `max_recommendations` και `corpus_type`. Στην `target_movie` περνάμε το ID μιας ταινίας-στόχου για την οποία μας ενδιαφέρει να βρούμε παρόμοιες ως προς το περιεχόμενο (τη σύνοψη) ταινίες, `max_recommendations` στο πλήθος.
Υλοποιήστε τη συνάρτηση ως εξής: 
- για την ταινία-στόχο, θα υπολογίζετε την [ομοιότητα συνημιτόνου](https://en.wikipedia.org/wiki/Cosine_similarity) της με όλες τις ταινίες της συλλογής σας όπως αυτές αναπαριστώνται στο `corpus_type`.
- με βάση την ομοιότητα συνημιτόνου που υπολογίσατε, δημιουργήστε ταξινομημένο πίνακα από το μεγαλύτερο στο μικρότερο, με τα indices (`ID`) των ταινιών. Παράδειγμα: αν η ταινία με index 1 έχει ομοιότητα συνημιτόνου με 3 ταινίες \[0.2 1 0.6\] (έχει ομοιότητα 1 με τον εαύτό της) ο ταξινομημένος αυτός πίνακας indices θα είναι \[1 2 0\].
- Για την ταινία-στόχο εκτυπώστε: id, τίτλο, σύνοψη, κατηγορίες (categories)
- Για τις `max_recommendations` ταινίες (πλην της ίδιας της ταινίας-στόχου που έχει cosine similarity 1 με τον εαυτό της) με τη μεγαλύτερη ομοιότητα συνημιτόνου (σε φθίνουσα σειρά), τυπώστε σειρά σύστασης (1 πιο κοντινή, 2 η δεύτερη πιο κοντινή κλπ), ομοιότητα συνημιτόνου, id, τίτλο, σύνοψη, και κατηγορίες (categories)


In [29]:
import numpy as np
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity


def content_recommender(target_movie, max_recommendations, corpus_type):
    sim = cosine_similarity(corpus_type)
    negatives = list(map(lambda x: -x, sim))
    ix = np.argsort(negatives[target_movie])[:max_recommendations+1]

    recommendations = [{'Similarity':sim[ID][target_movie], 'ID':ID, 'Title':titles[ID], 'Description':corpus[ID], 'Categories':categories[ID]} for ID in ix]
    df = pd.DataFrame(recommendations)
    movie = recommendations[0]
    print(f'ID: {movie["ID"]} \nTitle: {movie["Title"]} \nDescription: {movie["Description"]} \nCategories: {movie["Categories"]}')
    display(Markdown(df.iloc[1:].to_markdown()))

content_recommender(567, 5, tf_idf_matrix)

ID: 567 
Title: 500 Miles North 
Description: James and John are estranged brothers. But that is changed when they reunite to satisfy their father's last wishes. 
Categories: "Drama"


|    |   Similarity |   ID | Title                               | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | Categories                                                                             |
|---:|-------------:|-----:|:------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------|
|  1 |     0.2947   | 4517 | Aalkkoottathil Thaniye              | Rajan is forced by the death of his father, a schoolteacher, to reunite with his estranged brother and cousin, Ammukutty.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | "Drama"                                                                                |
|  2 |     0.246641 | 3737 | Lennon Naked                        | In 1964, in the peak of Beatlemania, a reluctant John Lennon  is persuaded by manager Brian Epstein  to meet Freddie Lennon , the father who abandoned him seventeen years earlier, with the press in attendance. When they meet, John accuses his father of abandoning him, but his father says that "he left it up to John." John and Brian quickly leave the meeting. The movie then jumps to 1967, after Brian Epstein has died. The Beatles are giving a press conference about their new film, Magical Mystery Tour. John is skeptical about the film, but Paul ([[Andrew Scott  convinces him to go through with the idea. John then invites his father to his mansion to live with him. Freddie Lennon arrives and meets his grandson, Julian. Sitting with his wife, John reads the criticism of Magical Mystery Tour, while comparing his wife to Brigitte Bardot, whom he says he will meet after he returns from India. John finds a letter addressed to him, with the word "Breathe" written on it. Later, after finding his father in a neighbor's house, Freddie reveals that he has a 19 year old girlfriend named Pauline, with whom he wants to live. Lennon accuses his father of leaving him again, and then leaves, after telling his father that he won't live with him anymore. After meeting Maharishi Mahesh Yogi, the Beatles quickly return to London, and in a press conference they say they made a mistake when they trusted Maharishi. The journalists are curious about the Beatles new business -- Apple Records. At night, before meeting Derek Taylor, John glimpses Yoko Ono from a window. John reveals to Derek that she sends him letters --one of them, the "Breathe" one. Derek and John go to meet Brigitte Bardot, and in the car John tells Derek he sometimes thinks he is Jesus Christ. Derek tells him that being John Lennon is enough to "walk on water". John is nervous about his meeting with Brigitte, and takes a tablet of LSD along with Derek. The next morning, John returns home, and during his drug-trip, he remembers the public's reaction at the "more popular than Jesus" statement he made two years before. He gets into his pool, and his wife Cynthia demands that he gets out. John says that he can't walk on water after all, and Cynthia and Julian leave. John stays with Pete Shotton in his mansion, and tells him he didn't do anything with Brigitte after all. He tells him to bring Yoko. Yoko arrives and records an album with Lennon, which would become Two Virgins. They spend the night together, and John tells Pete that he wants to live with Yoko. Some time later, Yoko and John hold a gallery event in which they set free various white balloons, and John says that "he's setting them free, including him". Cynthia demands that John stop his adultery with Yoko. Cynthia is willing to forgive John, but John doesn't care and chooses Yoko. While John is leaving, Julian is seen playing with a ball all alone, and he throws the ball at John. John, however, throws it away and doesn't pay attention to his son. All through the film, John has memories about himself in the Blackpool docks, when he was six years all. In a meeting with the Beatles, Derek is worried about releasing a double album, because Apple is losing money. After losing patience with Paul, John and Yoko leave the meeting and go to their house, and take a picture of what would be the cover of Two Virgins. Yoko reveals that she's pregnant, and John is very happy. The police later arrive because of charges of drug possession, and Yoko loses the baby. John asks Yoko to marry him. After a press conference, John reveals to Yoko that his father has had another child - David. John is later seen going through heroin withdrawal, and tells Pete that Yoko is pregnant again and he needs to clean up the place. Pete tells John that John must do it himself, but John refuses, so Pete leaves him. Yoko later loses the baby again. In a meeting with the Beatles, John announces to his partners that the band is over. Paul convinces John not to tell the press, and John calmly leaves. In 1970 Paul announces that the band is over, and John becomes angry about it, saying that Paul wanted to finish the band himself. John throws rocks at Paul's house, to the shock of the fans outside. John is later seen with a therapist, Art, who has him remember when he was six years old - the day that he was at Blackpool with his parents. In his memories, John sees his parents deciding who will keep him, and his father has John choose for himself. John chooses his father. However, after seeing his mother leave, he runs after her and his father leaves. A traumatized John recounts how he shouted to his father to go with them, and how his mother told him that he was not going to live with her, but with her aunt Mimi. John's father is later reunited with John, when he wants to create a biography about his life. John presents him with one of his latest songs - Mother. He loses patience with his father once again, accusing of him never visiting. He leaves his father. At a press conference, John and Yoko announce that they will go to New York to live. The final scene shows John and Yoko getting on a plane and flying away, while a text on screen reveals that after they left in 1971, they never returned. | "Biographical film"                                                                    |
|  3 |     0.210317 | 1986 | Iodine                              | John Clem  heads north to investigate the disappearance of his estranged father. Upon arriving at the family cottage, John runs into Avery , his father’s colleague. John receives no help on his dad’s disappearance, but instead, a new set of eyes in which to view the living world around him. Avery assures John that they had been working on a special project which involved using iodine as a water purification tool, to preserve their natural surroundings. Environmental concerns and future implications of modernized society become a share interest and source of impassioned philosophical discussion for the two and they bond. However, John being a man of immense insecurity and uncertainty is forced to endure isolating surroundings while on the search for his father. The pressure from the rest of the family causes John an emotional breakdown as he is unsure of whether he can trust Avery.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | "Mystery",  "Science Fiction",  "Drama"                                                |
|  4 |     0.206111 | 1858 | Krippendorf's Tribe                 | Respected anthropologist James Krippendorf  and his wife, Jennifer ([[Barbara Williams , bring their three children along during their much-enjoyed search in New Guinea for a lost tribe. The search fails, despite the family's best efforts. After Jennifer's death back in the U.S., James falls into academic stagnation, having spent all his foundation grant money raising the children as a single parent. Scheduled to lecture at a college and fearful of being charged with misuse of grant funds, James concocts an imaginary tribe, the Shelmikedmu, using the names of his children as a basis. He later fakes a 16&nbsp;mm "documentary" film, casting his children as tribe members and superimposing footage of a legitimate New Guinean tribe so as to enhance the illusion. Anthropologist Veronica Micelli  contacts cable-TV producer Henry Spivey , forcing James to continue creating fraudulent footage as James' rival Ruth Allen  becomes suspicious. Because he has described a culture unlike any other, Krippendorf's fraud becomes increasingly famous. James himself masquerades as a tribal elder, while his two sons, Mickey ([[Gregory Smith  and Edmund , create and enact increasingly imaginative rituals. Only the eldest child, James' daughter Shelly, refuses to participate due to her disgust at the dishonesty perpetrated by her father. Taking advantage of her curiosity, James tricks Veronica into participating in his false documentary. When she discovers the truth, she is initially angry, but later helps James continue his fraud. Ruth Allen travels to New Guinea, discovering no tribe in the location specified by James. She transmits the news via fax to a colleague, who exposes James at a gala. James' imaginative son, Mickey, improvises a lie, that the Shelmikedmu hide by means of a magical ritual known only to them. Unknown to the majority of the characters, Shelly has contacted the New Guineans befriended by her family during the futile search for the lost tribe, urging them to masquerade as the Shelmikedmu in order to disappoint Ruth Allen. The ruse succeeds, and the accusation of fraud is abandoned. James, relieved of his worries, ends his fraud. Because Veronica has become sexually involved with him during her participation in his deceit, she assumes the role of a mother toward the children, though she is not explicitly said to marry James.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | "Comedy of Errors",  "Family Film",  "Screwball comedy",  "Comedy",  "Film adaptation" |
|  5 |     0.196172 | 2106 | Sarah, Plain and Tall: Winter's End | Sarah and Jacob Witting are on their farm with their family. Jacob's estranged father John reappears one winter, wanting to make amends with his son.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | "Drama"                                                                                |

## Βελτιστοποίηση του TfidfVectorizer

Αφού υλοποιήσετε τη συνάρτηση `content_recommender` χρησιμοποιήστε την για να βελτιστοποιήσετε την `TfidfVectorizer`. Συγκεκριμένα, αρχικά μπορείτε να δείτε τι επιστρέφει το σύστημα για τυχαίες ταινίες-στόχους και για ένα μικρό `max_recommendations` (2 ή 3). Αν σε κάποιες ταινίες το σύστημα μοιάζει να επιστρέφει σημασιολογικά κοντινές ταινίες σημειώστε το `ID` τους. Δοκιμάστε στη συνέχεια να βελτιστοποιήσετε την `TfidfVectorizer` για τα συγκεκριμένα `ID` ώστε να επιστρέφονται σημασιολογικά κοντινές ταινίες για μεγαλύτερο αριθμό `max_recommendations`. Παράλληλα, όσο βελτιστοποιείτε την `TfidfVectorizer`, θα πρέπει να λαμβάνετε καλές συστάσεις για μεγαλύτερο αριθμό τυχαίων ταινιών. 

Ταυτόχρονα, μια αντίρροπη κατά κάποιο τρόπο κατεύθυνση της βελτιστοποίησης είναι να χρησιμοποιείτε τις παραμέτρους του `TfidfVectorizer` έτσι ώστε να μειώνονται οι διαστάσεις του Vector Space Model μέχρι το σημείο που θα αρχίσει να εμφανίζονται επιπτώσεις στην ποιότητα των συστάσεων. 




In [30]:
import joblib

try :
    opt_tf_idf = joblib.load('joblib/opt_tf_idf.pkl')
except:
    opt_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.1, max_features=500000, ngram_range=(1,3))
    opt_tf_idf = opt_vectorizer.fit_transform(final_corpus)
    joblib.dump(opt_tf_idf, 'joblib/opt_tf_idf.pkl')

print(opt_tf_idf.shape)

content_recommender(567, 5, opt_tf_idf)


(5000, 500000)
ID: 567 
Title: 500 Miles North 
Description: James and John are estranged brothers. But that is changed when they reunite to satisfy their father's last wishes. 
Categories: "Drama"


|    |   Similarity |   ID | Title                  | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Categories                                                                             |
|---:|-------------:|-----:|:-----------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------|
|  1 |    0.24141   | 4517 | Aalkkoottathil Thaniye | Rajan is forced by the death of his father, a schoolteacher, to reunite with his estranged brother and cousin, Ammukutty.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | "Drama"                                                                                |
|  2 |    0.125434  | 3737 | Lennon Naked           | In 1964, in the peak of Beatlemania, a reluctant John Lennon  is persuaded by manager Brian Epstein  to meet Freddie Lennon , the father who abandoned him seventeen years earlier, with the press in attendance. When they meet, John accuses his father of abandoning him, but his father says that "he left it up to John." John and Brian quickly leave the meeting. The movie then jumps to 1967, after Brian Epstein has died. The Beatles are giving a press conference about their new film, Magical Mystery Tour. John is skeptical about the film, but Paul ([[Andrew Scott  convinces him to go through with the idea. John then invites his father to his mansion to live with him. Freddie Lennon arrives and meets his grandson, Julian. Sitting with his wife, John reads the criticism of Magical Mystery Tour, while comparing his wife to Brigitte Bardot, whom he says he will meet after he returns from India. John finds a letter addressed to him, with the word "Breathe" written on it. Later, after finding his father in a neighbor's house, Freddie reveals that he has a 19 year old girlfriend named Pauline, with whom he wants to live. Lennon accuses his father of leaving him again, and then leaves, after telling his father that he won't live with him anymore. After meeting Maharishi Mahesh Yogi, the Beatles quickly return to London, and in a press conference they say they made a mistake when they trusted Maharishi. The journalists are curious about the Beatles new business -- Apple Records. At night, before meeting Derek Taylor, John glimpses Yoko Ono from a window. John reveals to Derek that she sends him letters --one of them, the "Breathe" one. Derek and John go to meet Brigitte Bardot, and in the car John tells Derek he sometimes thinks he is Jesus Christ. Derek tells him that being John Lennon is enough to "walk on water". John is nervous about his meeting with Brigitte, and takes a tablet of LSD along with Derek. The next morning, John returns home, and during his drug-trip, he remembers the public's reaction at the "more popular than Jesus" statement he made two years before. He gets into his pool, and his wife Cynthia demands that he gets out. John says that he can't walk on water after all, and Cynthia and Julian leave. John stays with Pete Shotton in his mansion, and tells him he didn't do anything with Brigitte after all. He tells him to bring Yoko. Yoko arrives and records an album with Lennon, which would become Two Virgins. They spend the night together, and John tells Pete that he wants to live with Yoko. Some time later, Yoko and John hold a gallery event in which they set free various white balloons, and John says that "he's setting them free, including him". Cynthia demands that John stop his adultery with Yoko. Cynthia is willing to forgive John, but John doesn't care and chooses Yoko. While John is leaving, Julian is seen playing with a ball all alone, and he throws the ball at John. John, however, throws it away and doesn't pay attention to his son. All through the film, John has memories about himself in the Blackpool docks, when he was six years all. In a meeting with the Beatles, Derek is worried about releasing a double album, because Apple is losing money. After losing patience with Paul, John and Yoko leave the meeting and go to their house, and take a picture of what would be the cover of Two Virgins. Yoko reveals that she's pregnant, and John is very happy. The police later arrive because of charges of drug possession, and Yoko loses the baby. John asks Yoko to marry him. After a press conference, John reveals to Yoko that his father has had another child - David. John is later seen going through heroin withdrawal, and tells Pete that Yoko is pregnant again and he needs to clean up the place. Pete tells John that John must do it himself, but John refuses, so Pete leaves him. Yoko later loses the baby again. In a meeting with the Beatles, John announces to his partners that the band is over. Paul convinces John not to tell the press, and John calmly leaves. In 1970 Paul announces that the band is over, and John becomes angry about it, saying that Paul wanted to finish the band himself. John throws rocks at Paul's house, to the shock of the fans outside. John is later seen with a therapist, Art, who has him remember when he was six years old - the day that he was at Blackpool with his parents. In his memories, John sees his parents deciding who will keep him, and his father has John choose for himself. John chooses his father. However, after seeing his mother leave, he runs after her and his father leaves. A traumatized John recounts how he shouted to his father to go with them, and how his mother told him that he was not going to live with her, but with her aunt Mimi. John's father is later reunited with John, when he wants to create a biography about his life. John presents him with one of his latest songs - Mother. He loses patience with his father once again, accusing of him never visiting. He leaves his father. At a press conference, John and Yoko announce that they will go to New York to live. The final scene shows John and Yoko getting on a plane and flying away, while a text on screen reveals that after they left in 1971, they never returned.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | "Biographical film"                                                                    |
|  3 |    0.108622  |    0 | The Big Empty          | John Person  is an out-of-work actor living in Los Angeles, with a credit card debt of over $27,000. Across the hall from him lives his friend, Grace. One night, his nebbish neighbor Neely, who wears a neck brace, invades his apartment with an unusual request: deliver a large blue suitcase to the truck stop of Baker, California, where it will be picked up by a man named "Cowboy", for which he will be given $25,000. John is also given a gun to defend the case. He initially refuses and thinks the plan is insane, but Neely has a file containing many of John's personal information, including masturbation preferences and sperm sample test results. Realizing that Neely is serious, John takes the job, but demands that his price be raised to exactly what he owes on his credit cards. Neely agrees, and John drives to Baker. Once there, he checks into the Royal Hawaiian Motel where he meets Elron, the peppy manager, who tells him that John has just missed meeting "Cowboy", and describes him as wearing a "big, stupid black duster and a black Stetson". Looking for a drink, John meets the bartender Stella, and is immediately held at gunpoint by the hot-tempered Randy, who thinks John is after his girlfriend. Later that night, a young girl named Ruthie comes to John's hotel room and gives back his wallet that he dropped in the bar, and they hit it off. It will later turn out that Ruth is Randy's girlfriend. The next day, John goes to a diner and meets Dan, who tells a lot of tall tales and conspiracy theories about what goes on in the desert. Later, John meets Ruthie outside a gas station, where he buys her some beer and Jack Daniels whisky, and a can of whipped cream "for later". Then, they drive out to "Devil's Crest", a dry lake bed far out of town. She describes how RVs and people "disappear without a trace" out there. The two then get drunk, mostly on a mixture of Jack Daniels and whipped cream. Ruthie gets sick and faints, so John drives her home to Stella's bar. The next day, the jealous Randy confronts John and asks what he and Ruthie did at Devil's Crest, and threatens to kill him if he even talks to Ruthie anymore. Later, at the bar, Stella reveals that she's not really Ruthie's mother; she found Ruthie wandering around the dry lake bed at Devil's Crest when she was two years old. John goes back to the motel, and hears that he missed Cowboy again, but that Cowboy left him a package: a bowling ball bag that he's supposed to hold on to, and is not allowed to open. Neely's name is on the name tag. Back at the hotel, Grace calls John, and says that Neely was murdered, and the FBI was snooping around, asking questions. She described the murder as Neely getting shot, then beheaded, and tells him Neely's head is missing. John immediately suspects Neely's head is in the bag, but can't be sure, since he can't open the bag. Candy, a hooker, comes into John's room, because she heard that John was meeting Cowboy. She previously had an "encounter" with Cowboy, and cautions John, because she has heard a rumor that Cowboy abducted three strippers from Las Vegas, and they were never seen again. She describes Cowboy's familiar black duster and Stetson, and John immediately becomes suspicious. John soon buries the bag, and meets an FBI agent named Agent Banks at the bar, who implicates him in Neely's murder and 75 mysterious disappearances related to the town of Baker. The next day, John finds that Randy has stolen his suitcase. He drives down to the junkyard, armed with his gun,and finds that Randy has tied Ruthie up. Randy and John have an armed standoff, but John convinces Randy to let him and Ruthie go by threatening to shoot Ruthie. Later, Ruthie comes to John's hotel room to tell him that Randy was arrested, and they make love. The following evening, Randy points a shotgun at John, and orders him to drive out to the desert and dig a hole. After the hole is dug, Randy is about to shoot him, but Randy is instead shot by Cowboy. It's time to deliver the suitcase and the bowling bag. Cowboy and John go back to the motel, and John finds more suitcases stacked in his room. Cowboy asks John to drive out to the dry lake bed at Devil's Crest with the suitcases, but John refuses, saying "he's done his job." Cowboy convinces John to take the suitcases by using Grace as a hostage. John goes to Devil's Crest and meets "Bob the Indian", who tells John to arrange the suitcases in a circle, then drives away. Soon, Cowboy arrives in an RV, with a group of travelers in blue tracksuits, similar to one Neely wore. One of the travelers is Ruthie. Cowboy opens the infamous bowling bag, and pulls out a pair of size 11 bowling shoes. He offers them to John, claiming that he will offer him a chance to "come with him" to Paradise. He refuses, so Cowboy instead gives the shoes to the barefooted Ruthie. John asks Ruthie about it, and she excitedly asks John to come with her. He declines, and then Cowboy shoots a flare into the air. John blacks out. Some time later, John wakes up, still in the middle of the desert, all alone. All the suitcases are open and empty, except for one in the middle of the circle, which is locked. Frustrated, John takes it, and walks to the highway, where Grace meets him, and says he has been missing for three days. She also has a key that Cowboy gave her, telling her that John "would know what to do with it". John opens the case, and finds the $27,000 originally promised to him. Back in LA, Agent Banks interrogates him about what happened in Baker, which explains the 75 disappearances attributed to Cowboy. Banks says that he can't bring a story like this to the families of those who have disappeared. He then tells John that his credit card debts were already paid off with money unrelated to the $27,000, which he thinks John won in Vegas. Banks hints that he believes John's story, and John sees a band-aid on Banks' neck, similar to one that appeared on John's neck after his encounter at Devil's Crest, which related to one of the conspiracy theories Dan told John about. Following all this, John and Grace are on a date at a bowling alley. Grace congratulates John for getting a supporting role in a movie, implying that John's acting prospects are becoming better. John, wearing size 11 shoes, bowls a ball down the alley, which is then shown rolling across the vast moonlit landscape of Devil's Crest. In the distance, a white flare rises.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | "Black comedy",  "Science Fiction",  "Comedy",  "Indie"                                |
|  4 |    0.10095   | 1858 | Krippendorf's Tribe    | Respected anthropologist James Krippendorf  and his wife, Jennifer ([[Barbara Williams , bring their three children along during their much-enjoyed search in New Guinea for a lost tribe. The search fails, despite the family's best efforts. After Jennifer's death back in the U.S., James falls into academic stagnation, having spent all his foundation grant money raising the children as a single parent. Scheduled to lecture at a college and fearful of being charged with misuse of grant funds, James concocts an imaginary tribe, the Shelmikedmu, using the names of his children as a basis. He later fakes a 16&nbsp;mm "documentary" film, casting his children as tribe members and superimposing footage of a legitimate New Guinean tribe so as to enhance the illusion. Anthropologist Veronica Micelli  contacts cable-TV producer Henry Spivey , forcing James to continue creating fraudulent footage as James' rival Ruth Allen  becomes suspicious. Because he has described a culture unlike any other, Krippendorf's fraud becomes increasingly famous. James himself masquerades as a tribal elder, while his two sons, Mickey ([[Gregory Smith  and Edmund , create and enact increasingly imaginative rituals. Only the eldest child, James' daughter Shelly, refuses to participate due to her disgust at the dishonesty perpetrated by her father. Taking advantage of her curiosity, James tricks Veronica into participating in his false documentary. When she discovers the truth, she is initially angry, but later helps James continue his fraud. Ruth Allen travels to New Guinea, discovering no tribe in the location specified by James. She transmits the news via fax to a colleague, who exposes James at a gala. James' imaginative son, Mickey, improvises a lie, that the Shelmikedmu hide by means of a magical ritual known only to them. Unknown to the majority of the characters, Shelly has contacted the New Guineans befriended by her family during the futile search for the lost tribe, urging them to masquerade as the Shelmikedmu in order to disappoint Ruth Allen. The ruse succeeds, and the accusation of fraud is abandoned. James, relieved of his worries, ends his fraud. Because Veronica has become sexually involved with him during her participation in his deceit, she assumes the role of a mother toward the children, though she is not explicitly said to marry James.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | "Comedy of Errors",  "Family Film",  "Screwball comedy",  "Comedy",  "Film adaptation" |
|  5 |    0.0822723 | 3953 | This Earth Is Mine     | In 1931, Lon Rambeau sends his daughter Elizabeth Rambeau away from London to Napa Valley, California to visit Lon's father Philippe Rambeau and Phillipe's sister Martha Fairon, owners of vast vineyards and a grand estate. Philippe and Martha welcome Elizabeth lavishly, then reveal the real reason for the celebration of her arrival: her betrothal, unknown to Elizabeth, between her and her cousin, Andre Swann. John Rambeau, obviously disliked by matriarch Martha, arrives at the party, flirts with Elizabeth and leads her out into the vineyards, where he mischievously reveals Rambeau family secrets: Philippe uses marriage to tighten the family hold on the valley; Phillipe's sister Martha and daughter Charlotte  had been married off to local landowners in order to increase the vineyard holdings &mdash; Phillipe now wants to marry off grand-daughter Elizabeth to Andre in order to absorb the Stag's Leap Winery . Continuing his explanation, John points out his house nearby, where his invalid mother Charlotte now lives, and says that he now knows that Martha's husband  is his real father, even though everyone claims that Charlotte's late husband had been the biological father to both John and his sister Monica. John then takes the speechless Elizabeth into his arms and kisses her, but she pulls away and runs into the house. The next day, John gets into an argument with Philippe by insisting that they ignore Prohibition laws and sell their grapes  to bootleggers. Philippe, who loves John, insists on remaining lawful, explaining once again how he will continue to cultivate and study wines until the time comes to sell them legally. John volunteers to show Elizabeth around the winery, bitterly pointing out the vast stores of wine going to waste. When he asks her about her past and tries to kiss her again, she once again pulls away. As they wander the estate, one of the female workers, Buz Dietrick, flirts with John. Back at the house, Martha spirits Elizabeth away to a lunch with Andre, who is quite comfortable with the idea of an arranged marriage. Meeting with the local association of grape growers, John tells them that he will make them rich by selling their grapes to a syndicate in Chicago. Some of the traditional growers insist on gaining Philippe's approval first. Ignoring them, John leaves for Chicago to meet with the syndicate &mdash; soon after, thugs arrive in the Napa Valley and force the growers to sign contracts. When John returns to Napa months later, the growers, now grown rich from John's arrangement with the Chicago syndicate, welcome him warmly. John meets the growers at a nightclub, where he finds Buz sitting with her boyfriend, Luigi Griffanti. They all watch Elizabeth and Andre  dance. John once again romances Elizabeth, who, hoping to dissuade his interest, reveals to him that her father had sent her away from England because she had been involved in a torrid love affair with a cruel man, and she now desires a safe marriage with Andre. A jealous John accuses her of "prostituting" herself for real estate, and she slaps him. John then turns to Buz for comfort, convincing her to leave Luigi and drive off with him. Later, Andre drives Elizabeth home, where he confesses that he is growing to love her. Inside, Martha, aware of Elizabeth's ambivalent feelings over John, confronts her about her past, urging her to marry Andre before he realizes he is getting "damaged goods." John returns home and tells his mother he will leave the next day for Chicago. The next day Elizabeth, having had a change of heart, races off to see John before his train leaves &mdash; she finds him at the station and tells him she loves him. Thrilled, John insists that she wait for his return. A few months later, a pregnant Buz shows up at the estate to inform Phillipe that the baby is John's &mdash; Buz and her father blackmail Philippe. Elizabeth, nearby, hears all, and collapses with grief. Martha directs Buz to tell Luigi the baby is his. Buz and Luigi soon marry, but she fights with both him and his mother when they want to name the baby Cesare, and she wants to name the baby ... John. Months pass. Martha tries to convince an impatient Andre to wait for Elizabeth to set a wedding date. When John finally returns to Napa Valley, Elizabeth is up in the mountain orchard with Philippe, listening to him explain that this is sacred ground to him, as it was his first plot of land, and his beloved wife is buried here. At the estate, Martha greets John coldly and warns him that he must now conduct business through her, because Philippe is ill. After successfully persuading Martha that they can make millions by selling the Rambeau grapes to the Chicago syndicate, John drives to the mountain orchard. There, Philippe, horrified by John's shady dealings, commands him to leave, as does Elizabeth, who tells him she knows the truth about Buz's baby. Protesting that he can prove that Buz has lied, John races away, accidentally throwing a lit cigarette into the parched fields, starting a blaze. By the time the fire is noticed, it is too late, and the out-of-control wildfire destroys most of the mountain orchard. Unaware of the fire, John rushes to Buz's house, where mother-in-law Griffanti, upon hearing John's name, figures out that John is her grandson's actual father. As Buz agrees that she will tell Elizabeth that the baby is not John's, Luigi, at his mother's urging, attacks John in a jealous rage. John manages to get away, but Luigi pulls out a gun and shoots him. At the hospital, John is diagnosed with temporary partial paralysis &mdash; when Elizabeth visits to beg his forgiveness, he turns away from her bitterly. Soon, he is able to walk with crutches, and returns home, where he tells his uncle Francis and mother Charlotte that he knows about their affair, which resulted in his birth. John takes the blame for the orchard fire &mdash; in return, he forgives the two of them for their deceit, and asks for their forgiveness in exchange. Months later, Philippe dies, and matriarch Martha gathers the family together for the reading of the will. To Martha's shock and dismay, Philippe has divided the estate equally between his children and grandchildren &mdash; he also leaves the mountain orchard to John, the valley vineyards to the still-unmarried Elizabeth, and nothing to Martha. Although Martha is greatly pained by what she sees as an injustice, both she and her husband Francis realize that now she will finally be more interested in her husband and her marriage, instead of the family business. Within a few weeks, John has restored his mountain orchard to health. One day, Elizabeth joins him with a gift: a grape vine cutting from her valley vineyard. They graft the valley cutting onto a mountain vine, talk about the melding of "valley sweetness" with "mountain strength", and, finally, fall into each other's arms, declaring their love for each other. | "Drama"                                                                                |

## Βαθιά μάθηση: δημιουργία corpora με χρήση word emmbeddings

Η προσέγγιση της κατασκευής μόνο μέσω tfidf του συστήματος συστάσεων έχει διάφορα μειονεκτήματα. Θα μας ενδιέφερε λοιπόν να δούμε αν μπορούμε να χρησιμοποιήσουμε για τις λέξεις **εμφυτεύματα (embeddings)**, δηλαδή τις πυκνές διανυσματικές αναπαραστάσεις για τις λέξεις που μας δίνει το μοντέλο **Word2Vec**

Ωστόσο, το dataset της κάθε ομάδας είναι πολύ μικρό για να εξάγουμε τα δικά μας word embeddings (και να είναι καλά). Για το λόγο αυτό θα χρησιμοποιήσουμε τη μεθοδολογία της Βαθιάς Μάθησης που είναι η **Μεταφορά Μάθησης (Transfer Learning).**.

Στη μεταφορά μάθησης ουσιαστικά μεταφέρουμε τη γνώση που έχει αποκτήσει ένα ήδη εκπαιδευμένο (και κατά κανόνα πολύ μεγάλο) σύστημα. Η μεταφορά γίνεται διαμέσου των τιμών των βαρών που έχει προσδιορίσει μετά το πέρας της εκπαίδευσης.

Στην περίπτωσή μας, δεν μας ενδιαφέρουν τόσο τα ίδια τα βάρη των μοντέλων από τα οποία θα κάνουμε μεταφορά μάθησης. Κάτι τέτοιο θα μας ενδιέφερε αν π.χ. θέλαμε να συνεχίσουμε την εκπαίδευση στα δικά μας κείμενα. Μας ενδιαφέρουν όμως τα ίδια τα εμφυτεύματα, δηλαδή τα embeddings (διανύσματα διαστάσεων $m$) που έχει μάθει το νευρωνικό για το λεξιλόγιο του (vocabulary). To vocabulary σε τέτοια μεγάλα νευρωνικά θα είναι πιθανότατα υπερσύνολο του δικού μας.

### Μεταφορά μάθησης εμφυτευμάτων



#### Εμφυτεύματα του Gensim-data
Το Gensim περιλαμβάνει αρκετά προεκπαιδευμένα μοντέλα εμφυτευμάτων Word2Vec. Με το επόμενο κελί παίρνουμε τη λίστα τους.

In [31]:
!pip install -U gensim
import gensim.downloader as api

print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


Τα μοντέλα αυτά βρίσκονται στο [αποθετήριο Gensim-data](https://github.com/RaRe-Technologies/gensim-data) όπου μπορείτε να βρείτε και την τεκμηρίωσή τους. Η φόρτωση των μοντέλων αυτών γίνεται με τη συνάρτηση `gensim.downloader.load`.

In [32]:
import joblib
try :
    twitter25 = joblib.load('joblib/twitter25.pkl')
except:
    twitter25 = api.load('glove-twitter-25') #1.2M vectors
    joblib.dump(twitter25, 'joblib/twitter25.pkl')

#### Άλλα εμφυτεύματα
Μπορείτε να βρείτε προεκπαιδευμένα εμφυτεύματα και από πηγές εκτός του Gensim. Για παράδειγμα:

- [Google News dataset](https://code.google.com/archive/p/word2vec/). Πρόκειται για προ-εκπαιδευμένα διανύσματα που έχουν εκπαιδευτεί σε μέρος του συνόλου δεδομένων Google News (περίπου 100 δισεκατομμύρια λέξεις). Το μοντέλο περιέχει διανύσματα 300 διαστάσεων για 3 εκατομμύρια λέξεις και φράσεις.
- [Amazon BlazingText](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). Το BlazingText δεν είναι μόνο προεκπαιδευμένα εμφυτεύματα αλλα και βελτιστοποιημένες υλοποιήσεις των αλγορίθμων Word2vec για την επεξεργασία κειμένου. Προυπόθεση είναι να δουλέψει κανείς στο SageMaker.

Οι διαδικασίες φόρτωσης embeddings από εξωτερικά δεδομένα μπορεί να είναι ελαφρά διαφορετικές από αυτή του Gensim.



#### Παρατηρήσεις

*   Επαναλαμβάνουμε ότι στην εργασία αυτή δεν μας ενδιαφέρουν τα ίδια τα μοντέλα αλλά το να μπορούμε για μία λέξη του λεξιλογίου μας να μπορούμε να βρούμε το embedding (διάνυσμα) που της αντιστοιχεί στο εκάστοτε προεκπαιδευμένο μοντέλο. 

*   Επίσης, δεν θα χρησιμοποιήσουμε την `Phrases` για να βρούμε bigrams στο dataset μας όπως θα ήταν το ορθότερο, καθώς αυτό θα απαιτούσε την συνέχιση της εκπαίδευσης του μοντέλου σε νέο λεξιλόγιο με πολύ λίγα νέα δεδομένα.


 ### Δημιουργία corpora βασισμένων στα εμφυτεύματα

Για να μπορέσουμε να ενσωματώσουμε τη γνώση που υπάρχει στα προεκπαιδευμένα εμφυτεύματα στο δικό μας corpus θα προχωρήσουμε όπως περιγράφεται ακολούθως.

Για κάθε περιγραφή ταινίας $d$, η οποία αποτελείται από τις $N_d$ λέξεις $w_i$, το  $tfidf$ της κάθε λέξης $w_i$ δίνεται από τη σχέση:

$$ tfidf(w_i) = tf(w_i,d) \cdot idf(w_i)$$

Ταυτόχρονα, σε κάθε λέξη $w_i$ αντιστοιχεί ένα διάνυσμα $W2V(w_i)$ από το μοντέλο εμφυτευμάτων που έχουμε εισάγει. Τα διανύσματα εμφυτευμάτων $W2V$ θα έχουν διάσταση $m$, ανάλογα το μοντέλο. 

Για κάθε ταινία d, μπορούμε να δημιουργήσουμε μια διανυσματική αναπαράσταση $W2V(d)$ διαστάσεων $m$ χρησιμοποιώντας το $tfidf(w_i)$ ως συντελεστή βαρύτητας για κάθε εμφύτευμα $W2V(w_i)$:

$$ W2V(d) = \frac{tfidf(w_1)\cdot W2V(w_i) + tfidf(w_2)\cdot W2V(w_2) + \dotsc  + tfidf(w_{N_{d}})\cdot W2V(w_{N_{d}})}{tfidf(w_1)+tfidf(w_2)+ \dotsc + tfidf(w_{N_{d}})}$$


#### build_tfw2v

Υλοποιήστε μια συνάρτηση `build_tfw2v` με ορίσματα:
- `corpus` που θα είναι το προεπεξεργασμένο dataset σας,
- `vectors` που θα είναι το μοντέλο που θα σας δίνει τα διανύσματα των εμφυτεύσεων vectors, και 
- `embeddings_size` που θα είναι η διάσταση των εμφυτευμάτων $m$.

H συνάρτηση αυτή θα επιστρέφει ένα νέο corpus που θα είναι ένας πίνακας 5000 (όσες οι ταινίες σας) x $m$ (το η διάσταση των εμφυτευμάτων). Ανάλογα ποιο μοντέλο χρησιμποιείτε για transfer learning ο πίνακας αυτός θα είναι διαφορετικός.

Μπορείτε πλεόν να καλείτε την `content_recommender` με διαφορετικά corpora στο όρισμα `corpus_type`. Σημειώστε ότι στο TFidfVectorizer χρησιμοποιουμε τη σειριακή μορφή των numpy arrays και ίσως σας χρησιμεύσει η `sparse.csr_matrix()` από την Scipy.

In [33]:
from scipy.sparse import csr_matrix

def build_tfw2v(corpus, vectors, embeddings_size):

    vectorizer = TfidfVectorizer(use_idf=True)
    tf_idf = vectorizer.fit_transform(corpus).toarray()
    vector_vocabulary = vectors.index_to_key
    tf_idf_vocabulary = vectorizer.vocabulary_
    w2v_corpus = np.zeros((len(corpus), embeddings_size))

    for index, description in enumerate(corpus):
        words = description.split()

        #For every word that exists in the vocabulary, multiply its vector by the TF-IDF of the word
        numerator = [vectors[word]*tf_idf[index][tf_idf_vocabulary[word]] for word in words if word in vector_vocabulary]

        #Matrix addition
        numerator = np.array(np.sum(numerator, axis=0))
        
        #Sum of all words' TF-IDF, existing in the vocabulary 
        denominator = np.sum([tf_idf[index][tf_idf_vocabulary[word]] for word in words if word in vector_vocabulary])

        w2v_desc = np.array(numerator/denominator)
        w2v_corpus[index] = w2v_desc

    return w2v_corpus

In [46]:
import joblib
try :
    corpora_twitter25 = joblib.load('joblib/corpora_twitter25.pkl')
except:
    corpora_twitter25 = build_tfw2v(pre_stem_corpus, twitter25, 25)
    joblib.dump(corpora_twitter25, 'joblib/corpora_twitter25.pkl')

In [35]:
content_recommender(567, 5, corpora_twitter25)

ID: 567 
Title: 500 Miles North 
Description: James and John are estranged brothers. But that is changed when they reunite to satisfy their father's last wishes. 
Categories: "Drama"


|    |   Similarity |   ID | Title            | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | Categories                                                                 |
|---:|-------------:|-----:|:-----------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------|
|  1 |     0.966405 | 3856 | Ashik Kerib      | The Ashik Kerib wants to marry his beloved, but her father opposes since Kerib is poor and he expects rich prospects for his ' daughter from heaven'. She vows to wait for him for a thousand days and nights until he comes back with enough money to impress her father. He sets out on a journey to gain wealth and encounters many difficulties, but with the help of a saint horseman, he returns to his beloved on the 1001st day and they are able to marry.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | "Romantic drama",  "Romance Film",  "Drama",  "Adventure",  "World cinema" |
|  2 |     0.958785 | 4347 | Saptaswaralu     | Devi Saraswathi gave Saraswati Peetham to a Gandharva and advises him to go some good to Bhoolokam. Shetfuygytpftlls in Mandakini river. Both the Gandharva and his lover are cursed to be born on Earth. Gandharva and the villain are in search of Sharada peetam. Finally the Gandharva procures the peetham and gives to an eligible old musician to care of it and utilize for good.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | "Drama"                                                                    |
|  3 |     0.957968 | 2653 | Samudhiram       | Selvarasu  is the head of the family and his two brothers  are ready to give up their life for him. The three brothers think the world of their sister Rasamani . A rich man  from the neighboring village feels insulted when Murali mishandles his son and to take revenge, makes Rasamani his daughter-in-law. Starting with taking away all of the brothers' wealth, he insults them at every opportunity while ill-treating Rasamani. At last in the climax Rasamani's husband and father-in-law reforms accepts her and her brothers.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | "Drama",  "World cinema"                                                   |
|  4 |     0.957189 | 2731 | Twelve Mile Road | Divorced farmer Stephen Landis  lives a normal life with his girlfriend Leah  and her daughter, Roxanne  until his daughter Dulcie  comes for a summer visit after having a fight with her mother . During the summer Dulcie starts to heal her problems with her parents and they get together again.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | "Drama"                                                                    |
|  5 |     0.955748 | 4961 | Padayappa        | Padayappa  is an engineer who visits his hometown to attend his sister's wedding. His sister is engaged to the son of his maternal uncle . During this happy occasion, Padayappa's father's foster brother  demands a share in the family property. However, standing true to his words that the family property should not be divided, Padayappa's father  gives the entire property to him. His family is then forced to leave their home. Unable to bear this shock, his father suddenly dies. His sister's wedding is then cancelled because the groom chose to marry someone offering him a greater dowry and married his uncle's daughter. Also, Nilambiri , the spoiled daughter of his maternal uncle, is in love with him. Meanwhile, Padayappa finds out that the hill that he owns is made of granite, and he digs it out and becomes famous in the ciy in the city as he uses the money in service of the poor. His family, once again, is able to settle down. His sister gets married to an engineer Padayappa hired, and Padayappa maintains his father's position in the village. Padayappa is in love with Vasundhara , Nilambari's servant. When Nilambari finds this out, she goes crazy. Her parents beg Padayappa's widowed mother to allow Nilambari to marry Padayappa. Padayappa obeys his mother's orders to get married. However, Padayappa's mother becomes wise and tells off her brother in front of the entire village, giving a proposal of marriage to Vasundhara's mother instead of her brother, not letting her brother forget the humiliation he put her through when Padayappa's father died. Nilambari's dad commits suicide in humiliation. Nilambari tries to kill Vasundhara, but is not successful. After the wedding, Nilambari retreats into a room in her brother's house, only having Padayappa in her mind for 18 years. However, when her brother finally wakes her up, she plans her revenge on Padayappa, now a father of two adult daughters. Nilambari's brother also has a son, Chandru Prakash  who studies in the same college as Padayappa's elder daughter . He is told by Nilambari that he should make Padayappa's daughter fall in love with him. At the same time Padayappa plans to get his eldest daughter married to his sister's son. As a tradition in their village the groom and bride are asked if they both are not being forced to marry each other. Nilambari's plan is to make Padayappa's daughter say that she doesn't wish to get married to the groom that her parents want her to marry and that she is in love with someone else and in that process get Padayappa humiliated. Her plan works, but Padayappa retaliates and makes his daughter's lover confess that he really fell in love with her, even though he was told only to act. Padayappa decides to take him to his house for the marriage, with Nilambari and her brother chasing behind. The chase sees Nilambari's brother killed in a car accident. After a dramatic fight scene, the marriage takes place. Nilambari reaches the temple where the marriage is conducted and in anger tries to kill Padayappa. But Padayappa succeeds in saving her life by preventing a bull from attacking her while dodging the bullets fired from the gun. Rather than live with the humiliation of knowing that she was unsuccessful in avenging her father's death and having her life saved by her enemy, Nilambari commits suicide. | "Romance Film",  "Drama"                                                   |

## Ανάλυση αποτελεσμάτων

### Σύστημα συστάσεων βασισμένο μόνο στο tfidf

- Σε markdown περιγράψτε τι προεπεξεργασία κάνετε στα κείμενα και γιατί.

- Περιγράψτε πως προχωρήσατε στις επιλογές σας για τη βελτιστοποίηση της `TfidfVectorizer`. 

- [Cherry-picking:](https://www.wikiwand.com/en/Cherry_picking) Δώσετε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν καλά αποτελέσματα μέχρι `max_recommendations` (τουλάχιστον 5) και σχολιάστε.

- [Nit-picking:](https://en.wikipedia.org/wiki/Nitpicking) Δώστε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν κακά αποτελέσματα και σχολιάστε.

- Ποια είναι συνολικά τα πλεονεκτήματα και μειονεκτήματα ενός recommender βασισμένου στο tfidf;

### Σύγκριση και σχολιασμός με recommenders βασισμένων στο Word2Vec

- Υλoποιήστε recommenders που βασίζονται σε μεταφορά μάθησης και εμφυτεύματα. Χρησιμοποιήστε παραδείγματα για να υποδείξετε δυνατά και αδύναμα σημεία τους.

- Μπορείτε να σχολιάσετε τα recommenders που βασίζονται στο Word2Vec σε σχέση με το απλό μοντέλο tfidf, εξετάζοντας τις συστάσεις για ίδια ID.

- Μπορείτε επίσης να εξετάσετε συγκριτικά τα Word2Vec recommenders μεταξύ τους και πάλι βασιζόμενοι σε παραδείγματα.

- Οι παρατηρήσεις σας θα βασίζονται στην ανάλυση των ποιοτικών χαρακτηριστικών που είναι η σειρά και το σύνολο των συστάσεων. Ωστόσο, μπορείτε να συμπεριλάβετε και ποσοτικά χαρακτηριστικά όπως τους χρονους loading και συγκρότησης του corpus αλλά και της διαστατικότητας $m$.

Χρησιμοποιήστε όποια μορφή reporting κρίνετε καταλληλότερη: κείμενο, πίνακες, διαγράμματα.


## Πρακτικό tip - persistence αντικειμένων με joblib.dump

Καθώς στην δεύτερη εργασία καλείστε να δημιουργήσετε διάφορα corpora των οποίων η δημιουργία παίρνει χρόνο, υπάρχει ένας εύκολος τρόπος να αποθηκεύουμε μεταβλητές σε dump files και να τις διαβάζουμε απευθείας.

H βιβλιοθήκη [joblib](https://pypi.python.org/pypi/joblib) της Python δίνει κάποιες εξαιρετικά χρήσιμες ιδιότητες στην ανάπτυξη κώδικα: pipelining, παραλληλισμό, caching και variable persistence. Τις τρεις πρώτες ιδιότητες τις είδαμε στην πρώτη άσκηση. Στην παρούσα άσκηση θα μας φανεί χρήσιμη η τέταρτη, το persistence των αντικειμένων. Συγκεκριμένα μπορούμε με:

```python
joblib.dump(my_object, 'my_object.pkl') 
```

να αποθηκεύσουμε οποιοδήποτε αντικείμενο-μεταβλητή (εδώ το `my_object`) απευθείας πάνω στο filesystem ως αρχείο, το οποίο στη συνέχεια μπορούμε να ανακαλέσουμε ως εξής:

```python
my_object = joblib.load('my_object.pkl')
```

Μπορούμε έτσι να ανακαλέσουμε μεταβλητές ακόμα και αφού κλείσουμε και ξανανοίξουμε το notebook, χωρίς να χρειαστεί να ακολουθήσουμε ξανά όλα τα βήματα ένα - ένα για την παραγωγή τους, κάτι ιδιαίτερα χρήσιμο αν αυτή η διαδικασία είναι χρονοβόρα.

Ας αποθηκεύσουμε το `corpus_tf_idf` και στη συνέχεια ας το ανακαλέσουμε.

In [36]:
import joblib

joblib.dump(corpora_twitter25, 'joblib/corpora_twitter25.pkl') 

['corpora_twitter25.pkl']



Μπορείτε με ένα απλό `!ls` να δείτε ότι το αρχείο `corpus_tf_idf.pkl` υπάρχει στο filesystem σας (== persistence):

In [37]:
!ls -lh

total 39912
-rw-r--r--  1 alex  staff   201K Dec 24 20:44 NN_Team_22_Unsupervised_Learning.ipynb
-rw-r--r--  1 alex  staff   977K Dec 26 11:12 corpora_twitter25.pkl
-rw-r--r--  1 alex  staff    12M Dec 26 11:05 opt_tf_idf.pkl
-rw-r--r--  1 alex  staff   6.3M Dec 23 17:10 tf_idf_matrix.pkl


και μπορούμε να τα διαβάσουμε με `joblib.load`

In [38]:
tf_idf_matrix = joblib.load('joblib/tf_idf_matrix.pkl')

# Εφαρμογή 2.  Τοπολογική και σημασιολογική απεικόνιση της ταινιών με χρήση SOM
<img src="https://i.imgur.com/Z4FdurD.jpg" width="60%">

## Δημιουργία dataset
Στη δεύτερη εφαρμογή θα βασιστούμε στις τοπολογικές ιδιότητες των Self Organizing Maps (SOM) για να φτιάξουμε ενά χάρτη (grid) δύο διαστάσεων όπου θα απεικονίζονται όλες οι ταινίες της συλλογής της ομάδας με τρόπο χωρικά συνεκτικό ως προς το περιεχόμενο και κυρίως το είδος τους (ο παραπάνω χάρτης είναι ενδεικτικός, δεν αντιστοιχεί στο dataset μας). 

Διαλέξτε για την αναπαράσταση των documents αυτήν που πιστεύετε απέδωσε καλύτερα στο πρώτα σκέλος της άσκησης. Έστω ότι αυτή είναι η `my_best_corpus`.

Η έτοιμη συνάρτηση `build_final_set` θα ενώσει την αναπαράσταση που θα της δώσετε ως όρισμα `mycorpus` με τις binarized κατηγορίες `catbins` των ταινιών ως επιπλέον κολόνες (χαρακτηριστικά). Συνεπώς, κάθε ταινία αναπαρίσταται στο Vector Space Model από τα χαρακτηριστικά της αναπαράστασης `mycorpus` και τις κατηγορίες της.

Τέλος, η συνάρτηση δέχεται ένα ορισμα για το πόσες ταινίες να επιστρέψει, με default τιμή όλες τις ταινίες (5000). Αυτό είναι χρήσιμο για να μπορείτε αν θέλετε να φτιάχνετε μικρότερα σύνολα δεδομένων ώστε να εκπαιδεύεται ταχύτερα το SOM. 

Θα τρέχουμε τη συνάρτηση με `final_set = build_final_set(my_best_corpus)`.

In [19]:
def build_final_set(mycorpus, doc_limit = 5000, tf_idf_only=False):
    # convert sparse tf_idf to dense tf_idf representation
    try:
        dense_tf_idf = mycorpus.toarray()[0:doc_limit,:]
    except:
        dense_tf_idf = mycorpus[0:doc_limit,:]
    if tf_idf_only:
        # use only tf_idf
        final_set = dense_tf_idf
    else:
        # append the binary categories features horizontaly to the (dense) tf_idf features
        final_set = np.hstack((dense_tf_idf, catbins[0:doc_limit,:]))
    # η somoclu θέλει δεδομ΄ένα σε float32
    return np.array(final_set, dtype=np.float32)


Στο επόμενο κελί, τυπώνουμε τις διαστάσεις του τελικού dataset μας. **Χωρίς βελτιστοποίηση του TFIDF** θα έχουμε περίπου 50.000 χαρακτηριστικά και ο θα είναι ανέφικτο να προχωρήσουμε στην εκπαίδευση του SOM.

In [67]:
import joblib

try :
    final_set = joblib.load('joblib/final_set.pkl')
except:
    final_set = build_final_set(corpora_twitter25)
    joblib.dump(final_set, 'joblib/final_set.pkl')


final_set.shape

(5000, 347)

## Εκπαίδευση χάρτη SOM

Θα δουλέψουμε με τη βιβλιοθήκη SOM ["Somoclu"](http://somoclu.readthedocs.io/en/stable/index.html). Εισάγουμε τις somoclu και matplotlib και λέμε στη matplotlib να τυπώνει εντός του notebook (κι όχι σε pop up window).

In [39]:
# install somoclu
# !pip install --upgrade somoclu
# import sompoclu, matplotlib
import somoclu
import matplotlib
# we will plot inside the notebook and not in separate window
%matplotlib inline

Καταρχάς διαβάστε το [function reference](http://somoclu.readthedocs.io/en/stable/reference.html) του somoclu. Θα δoυλέψουμε με χάρτη τύπου planar, παραλληλόγραμμου σχήματος νευρώνων με τυχαία αρχικοποίηση (όλα αυτά είναι default). Μπορείτε να δοκιμάσετε διάφορα μεγέθη χάρτη ωστόσο όσο ο αριθμός των νευρώνων μεγαλώνει, μεγαλώνει και ο χρόνος εκπαίδευσης. Για το training δεν χρειάζεται να ξεπεράσετε τα 100 epochs. Σε γενικές γραμμές μπορούμε να βασιστούμε στις default παραμέτρους μέχρι να έχουμε τη δυνατότητα να οπτικοποιήσουμε και να αναλύσουμε ποιοτικά τα αποτελέσματα. Ξεκινήστε με ένα χάρτη 10 x 10, 100 epochs training και ένα υποσύνολο των ταινιών (π.χ. 2000). Χρησιμοποιήστε την `time` για να έχετε μια εικόνα των χρόνων εκπαίδευσης. 

In [69]:
import time

som = somoclu.Somoclu(10, 10, maptype="planar", data=final_set)
start_time = time.time()
som.train(epochs=100)
train_time = time.time() - start_time

In [77]:
print(som.codebook)

[[-0.5356946   0.10631939  0.31800568 ...  0.          0.
   0.        ]
 [-0.42724928  0.20112456  0.12360832 ...  0.          0.
   0.        ]
 [-0.3690443   0.26252583  0.05632065 ...  0.          0.
   0.        ]
 ...
 [-0.46952793  0.06281371  0.31120095 ...  0.          0.
   0.05242496]
 [-0.3914141   0.11627609  0.20611934 ...  0.00567755  0.
   0.00567755]
 [-0.32972148  0.23270175  0.08093315 ...  0.03287627  0.
   0.        ]]



## Best matching units

Μετά από κάθε εκπαίδευση αποθηκεύστε σε μια μεταβλητή τα best matching units (bmus) για κάθε ταινία. Τα bmus μας δείχνουν σε ποιο νευρώνα ανήκει η κάθε ταινία. **Προσοχή: η σύμβαση των συντεταγμένων των νευρώνων στη Somoclu είναι (στήλη, γραμμή) δηλαδή το ανάποδο από την Python**. Με χρήση της [np.unique](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.unique.html) (μια πολύ χρήσιμη συνάρτηση στην άσκηση) αποθηκεύστε τα μοναδικά best matching units και τους δείκτες τους (indices) προς τις ταινίες. 

Σημειώστε ότι μπορεί να έχετε λιγότερα μοναδικά bmus από αριθμό νευρώνων γιατί μπορεί σε κάποιους νευρώνες να μην έχουν ανατεθεί ταινίες. Ως αριθμό νευρώνα θα θεωρήσουμε τον αριθμό γραμμής στον πίνακα μοναδικών bmus.


In [70]:
bmus = som.bmus
print(bmus.shape)
unique_bmus, indices = np.unique(bmus, return_index=True, axis=0)

(5000, 2)



## Ομαδοποίηση (clustering)

Τυπικά, η ομαδοποίηση σε ένα χάρτη SOM προκύπτει από το unified distance matrix (U-matrix): για κάθε κόμβο υπολογίζεται η μέση απόστασή του από τους γειτονικούς κόμβους. Εάν χρησιμοποιηθεί μπλε χρώμα στις περιοχές του χάρτη όπου η τιμή αυτή είναι χαμηλή (μικρή απόσταση) και κόκκινο εκεί που η τιμή είναι υψηλή (μεγάλη απόσταση), τότε μπορούμε να πούμε ότι οι μπλε περιοχές αποτελούν clusters και οι κόκκινες αποτελούν σύνορα μεταξύ clusters.

To somoclu δίνει την επιπρόσθετη δυνατότητα να κάνουμε ομαδοποίηση των νευρώνων χρησιμοποιώντας οποιονδήποτε αλγόριθμο ομαδοποίησης του scikit-learn. Στην άσκηση θα χρησιμοποιήσουμε τον k-Means. Για τον αρχικό σας χάρτη δοκιμάστε ένα k=20 ή 25. Οι δύο προσεγγίσεις ομαδοποίησης είναι διαφορετικές, οπότε περιμένουμε τα αποτελέσματα να είναι κοντά αλλά όχι τα ίδια.


In [89]:
print(som.codebook)

[[-0.5356946   0.10631939  0.31800568 ...  0.          0.
   0.        ]
 [-0.42724928  0.20112456  0.12360832 ...  0.          0.
   0.        ]
 [-0.3690443   0.26252583  0.05632065 ...  0.          0.
   0.        ]
 ...
 [-0.46952793  0.06281371  0.31120095 ...  0.          0.
   0.05242496]
 [-0.3914141   0.11627609  0.20611934 ...  0.00567755  0.
   0.00567755]
 [-0.32972148  0.23270175  0.08093315 ...  0.03287627  0.
   0.        ]]


In [90]:
from sklearn.cluster import KMeans

som.cluster(KMeans(n_clusters=20))

/Users/alex/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'split'


## Αποθήκευση του SOM

Επειδή η αρχικοποίηση του SOM γίνεται τυχαία και το clustering είναι και αυτό στοχαστική διαδικασία, οι θέσεις και οι ετικέτες των νευρώνων και των clusters θα είναι διαφορετικές κάθε φορά που τρέχετε τον χάρτη, ακόμα και με τις ίδιες παραμέτρους. Για να αποθηκεύσετε ένα συγκεκριμένο som και clustering χρησιμοποιήστε και πάλι την `joblib`. Μετά την ανάκληση ενός SOM θυμηθείτε να ακολουθήσετε τη διαδικασία για τα bmus.


In [47]:
joblib.dump(som, 'joblib/som.pkl')

['joblib/som.pkl']


## Οπτικοποίηση U-matrix, clustering και μέγεθος clusters

Για την εκτύπωση του U-matrix χρησιμοποιήστε τη `view_umatrix` με ορίσματα `bestmatches=True` και `figsize=(15, 15)` ή `figsize=(20, 20)`. Τα διαφορετικά χρώματα που εμφανίζονται στους κόμβους αντιπροσωπεύουν τα διαφορετικά clusters που προκύπτουν από τον k-Means. Μπορείτε να εμφανίσετε τη λεζάντα του U-matrix με το όρισμα `colorbar`. Μην τυπώνετε τις ετικέτες (labels) των δειγμάτων, είναι πολύ μεγάλος ο αριθμός τους.

Για μια δεύτερη πιο ξεκάθαρη οπτικοποίηση του clustering τυπώστε απευθείας τη μεταβλητή `clusters`.

Τέλος, χρησιμοποιώντας πάλι την `np.unique` (με διαφορετικό όρισμα) και την `np.argsort` (υπάρχουν και άλλοι τρόποι υλοποίησης) εκτυπώστε τις ετικέτες των clusters (αριθμοί από 0 έως k-1) και τον αριθμό των νευρώνων σε κάθε cluster, με φθίνουσα ή αύξουσα σειρά ως προς τον αριθμό των νευρώνων. Ουσιαστικά είναι ένα εργαλείο για να βρίσκετε εύκολα τα μεγάλα και μικρά clusters. 

Ακολουθεί ένα μη βελτιστοποιημένο παράδειγμα για τις τρεις προηγούμενες εξόδους:

<img src="https://image.ibb.co/i0tsfR/umatrix_s.jpg" width="35%">
<img src="https://image.ibb.co/nLgHEm/clusters.png" width="35%">




## Σημασιολογική ερμηνεία των clusters

Προκειμένου να μελετήσουμε τις τοπολογικές ιδιότητες του SOM και το αν έχουν ενσωματώσει σημασιολογική πληροφορία για τις ταινίες διαμέσου της διανυσματικής αναπαράστασης του tf-idf, των εμφυτευμάτων και των κατηγοριών, χρειαζόμαστε ένα κριτήριο ποιοτικής επισκόπησης των clusters. 

Θα υλοποιήσουμε το εξής κριτήριο: Λαμβάνουμε όρισμα έναν αριθμό (ετικέτα) cluster. Για το cluster αυτό βρίσκουμε όλους τους νευρώνες που του έχουν ανατεθεί από τον k-Means. Για όλους τους νευρώνες αυτούς βρίσκουμε όλες τις ταινίες που τους έχουν ανατεθεί (για τις οποίες αποτελούν bmus). Για όλες αυτές τις ταινίες τυπώνουμε ταξινομημένη τη συνολική στατιστική όλων των ειδών (κατηγοριών) και τις συχνότητές τους. Αν το cluster διαθέτει καλή συνοχή και εξειδίκευση, θα πρέπει κάποιες κατηγορίες να έχουν σαφώς μεγαλύτερη συχνότητα από τις υπόλοιπες. Θα μπορούμε τότε να αναθέσουμε αυτήν/ές την/τις κατηγορία/ες ως ετικέτες κινηματογραφικού είδους στο cluster.

Μπορείτε να υλοποιήσετε τη συνάρτηση αυτή όπως θέλετε. Μια πιθανή διαδικασία θα μπορούσε να είναι η ακόλουθη:

1. Ορίζουμε συνάρτηση `print_categories_stats` που δέχεται ως είσοδο λίστα με ids ταινιών. Δημιουργούμε μια κενή λίστα συνολικών κατηγοριών. Στη συνέχεια, για κάθε ταινία επεξεργαζόμαστε το string `categories` ως εξής: δημιουργούμε μια λίστα διαχωρίζοντας το string κατάλληλα με την `split` και αφαιρούμε τα whitespaces μεταξύ ετικετών με την `strip`. Προσθέτουμε τη λίστα αυτή στη συνολική λίστα κατηγοριών με την `extend`. Τέλος χρησιμοποιούμε πάλι την `np.unique` για να μετρήσουμε συχνότητα μοναδικών ετικετών κατηγοριών και ταξινομούμε με την `np.argsort`. Τυπώνουμε τις κατηγορίες και τις συχνότητες εμφάνισης ταξινομημένα. Χρήσιμες μπορεί να σας φανούν και οι `np.ravel`, `np.nditer`, `np.array2string` και `zip`.

2. Ορίζουμε τη βασική μας συνάρτηση `print_cluster_neurons_movies_report` που δέχεται ως όρισμα τον αριθμό ενός cluster. Με τη χρήση της `np.where` μπορούμε να βρούμε τις συντεταγμένες των bmus που αντιστοιχούν στο cluster και με την `column_stack` να φτιάξουμε έναν πίνακα bmus για το cluster. Προσοχή στη σειρά (στήλη - σειρά) στον πίνακα bmus. Για κάθε bmu αυτού του πίνακα ελέγχουμε αν υπάρχει στον πίνακα μοναδικών bmus που έχουμε υπολογίσει στην αρχή συνολικά και αν ναι προσθέτουμε το αντίστοιχο index του νευρώνα σε μια λίστα. Χρήσιμες μπορεί να είναι και οι `np.rollaxis`, `np.append`, `np.asscalar`. Επίσης πιθανώς να πρέπει να υλοποιήσετε ένα κριτήριο ομοιότητας μεταξύ ενός bmu και ενός μοναδικού bmu από τον αρχικό πίνακα bmus.

3. Υλοποιούμε μια βοηθητική συνάρτηση `neuron_movies_report`. Λαμβάνει ένα σύνολο νευρώνων από την `print_cluster_neurons_movies_report` και μέσω της `indices` φτιάχνει μια λίστα με το σύνολο ταινιών που ανήκουν σε αυτούς τους νευρώνες. Στο τέλος καλεί με αυτή τη λίστα την `print_categories_stats` που τυπώνει τις στατιστικές των κατηγοριών.

Μπορείτε βέβαια να προσθέσετε οποιαδήποτε επιπλέον έξοδο σας βοηθάει. Μια χρήσιμη έξοδος είναι πόσοι νευρώνες ανήκουν στο cluster και σε πόσους και ποιους από αυτούς έχουν ανατεθεί ταινίες.

Θα επιτελούμε τη σημασιολογική ερμηνεία του χάρτη καλώντας την `print_cluster_neurons_movies_report` με τον αριθμός ενός cluster που μας ενδιαφέρει. 

Παράδειγμα εξόδου για ένα cluster (μη βελτιστοποιημένος χάρτης, ωστόσο βλέπετε ότι οι μεγάλες κατηγορίες έχουν σημασιολογική  συνάφεια):

```
Overall Cluster Genres stats:  
[('"Horror"', 86), ('"Science Fiction"', 24), ('"B-movie"', 16), ('"Monster movie"', 10), ('"Creature Film"', 10), ('"Indie"', 9), ('"Zombie Film"', 9), ('"Slasher"', 8), ('"World cinema"', 8), ('"Sci-Fi Horror"', 7), ('"Natural horror films"', 6), ('"Supernatural"', 6), ('"Thriller"', 6), ('"Cult"', 5), ('"Black-and-white"', 5), ('"Japanese Movies"', 4), ('"Short Film"', 3), ('"Drama"', 3), ('"Psychological thriller"', 3), ('"Crime Fiction"', 3), ('"Monster"', 3), ('"Comedy"', 2), ('"Western"', 2), ('"Horror Comedy"', 2), ('"Archaeology"', 2), ('"Alien Film"', 2), ('"Teen"', 2), ('"Mystery"', 2), ('"Adventure"', 2), ('"Comedy film"', 2), ('"Combat Films"', 1), ('"Chinese Movies"', 1), ('"Action/Adventure"', 1), ('"Gothic Film"', 1), ('"Costume drama"', 1), ('"Disaster"', 1), ('"Docudrama"', 1), ('"Film adaptation"', 1), ('"Film noir"', 1), ('"Parody"', 1), ('"Period piece"', 1), ('"Action"', 1)]```
   


## Tips για το SOM και το clustering

- Για την ομαδοποίηση ένα U-matrix καλό είναι να εμφανίζει και μπλε-πράσινες περιοχές (clusters) και κόκκινες περιοχές (ορίων). Παρατηρήστε ποια σχέση υπάρχει μεταξύ αριθμού ταινιών στο final set, μεγέθους grid και ποιότητας U-matrix.
- Για το k του k-Means προσπαθήστε να προσεγγίζει σχετικά τα clusters του U-matrix (όπως είπαμε είναι διαφορετικοί μέθοδοι clustering). Μικρός αριθμός k δεν θα σέβεται τα όρια. Μεγάλος αριθμός θα δημιουργεί υπο-clusters εντός των clusters που φαίνονται στο U-matrix. Το τελευταίο δεν είναι απαραίτητα κακό, αλλά μεγαλώνει τον αριθμό clusters που πρέπει να αναλυθούν σημασιολογικά.
- Σε μικρούς χάρτες και με μικρά final sets δοκιμάστε διαφορετικές παραμέτρους για την εκπαίδευση του SOM. Σημειώστε τυχόν παραμέτρους που επηρεάζουν την ποιότητα του clustering για το dataset σας ώστε να τις εφαρμόσετε στους μεγάλους χάρτες.
- Κάποια τοπολογικά χαρακτηριστικά εμφανίζονται ήδη σε μικρούς χάρτες. Κάποια άλλα χρειάζονται μεγαλύτερους χάρτες. Δοκιμάστε μεγέθη 20x20, 25x25 ή και 30x30 και αντίστοιχη προσαρμογή των k. Όσο μεγαλώνουν οι χάρτες, μεγαλώνει η ανάλυση του χάρτη αλλά μεγαλώνει και ο αριθμός clusters που πρέπει να αναλυθούν.




## Ανάλυση τοπολογικών ιδιοτήτων χάρτη SOM

Μετά το πέρας της εκπαίδευσης και του clustering θα έχετε ένα χάρτη με τοπολογικές ιδιότητες ως προς τα είδη των ταίνιών της συλλογής σας, κάτι αντίστοιχο με την εικόνα στην αρχή της Εφαρμογής 2 αυτού του notebook. Η συγκεκριμένη εικόνα είναι μόνο για εικονογράφιση, δεν είναι χάρτης SOM καιδεν έχει καμία σχέση με τη συλλογή δεδομένων και τις κατηγορίες μας.

Για τον τελικό χάρτη SOM που θα παράξετε για τη συλλογή σας, αναλύστε σε markdown με συγκεκριμένη αναφορά σε αριθμούς clusters και τη σημασιολογική ερμηνεία τους τις εξής τρεις τοπολογικές ιδιότητες του SOM: 

1. Δεδομένα που έχουν μεγαλύτερη πυκνότητα πιθανότητας στο χώρο εισόδου τείνουν να απεικονίζονται με περισσότερους νευρώνες στο χώρο μειωμένης διαστατικότητας. Δώστε παραδείγματα από συχνές και λιγότερο συχνές κατηγορίες ταινιών. Χρησιμοποιήστε τις στατιστικές των κατηγοριών στη συλλογή σας και τον αριθμό κόμβων που χαρακτηρίζουν.
2. Μακρινά πρότυπα εισόδου τείνουν να απεικονίζονται απομακρυσμένα στο χάρτη. Υπάρχουν χαρακτηριστικές κατηγορίες ταινιών που ήδη από μικρούς χάρτες τείνουν να τοποθετούνται σε διαφορετικά ή απομονωμένα σημεία του χάρτη.
3. Κοντινά πρότυπα εισόδου τείνουν να απεικονίζονται κοντά στο χάρτη. Σε μεγάλους χάρτες εντοπίστε είδη ταινιών και κοντινά τους υποείδη.

Προφανώς τοποθέτηση σε 2 διαστάσεις που να σέβεται μια απόλυτη τοπολογία δεν είναι εφικτή, αφενός γιατί δεν υπάρχει κάποια απόλυτη εξ ορισμού για τα κινηματογραφικά είδη ακόμα και σε πολλές διαστάσεις, αφετέρου γιατί πραγματοποιούμε μείωση διαστατικότητας.

Εντοπίστε μεγάλα clusters και μικρά clusters που δεν έχουν σαφή χαρακτηριστικά. Εντοπίστε clusters συγκεκριμένων ειδών που μοιάζουν να μην έχουν τοπολογική συνάφεια με γύρω περιοχές. Προτείνετε πιθανές ερμηνείες.


Τέλος, εντοπίστε clusters που έχουν κατά την άποψή σας ιδιαίτερο ενδιαφέρον στη συλλογή της ομάδας σας (data exploration / discovery value) και σχολιάστε.



# Τελική παράδοση άσκησης

- Θα παραδώσετε στο helios το παρόν notebook επεξεργασμένο ή ένα ή δύο νέα zipαρισμένα με τις απαντήσεις σας για τα ζητούμενα και των δύο εφαρμογών. 
- Θυμηθείτε ότι η ανάλυση του χάρτη στο markdown με αναφορά σε αριθμούς clusters πρέπει να αναφέρεται στον τελικό χάρτη με τα κελιά ορατά που θα παραδώσετε αλλιώς ο χάρτης που θα προκύψει θα είναι διαφορετικός και τα labels των clusters δεν θα αντιστοιχούν στην ανάλυσή σας. 
- Μην ξεχάσετε στην αρχή ένα κελί markdown με **τα στοιχεία της ομάδας σας**.

<table>
  <tr><td align="center">
    <font size="4">Παρακαλούμε διατρέξτε βήμα-βήμα το notebook για να μην ξεχάσετε παραδοτέα</font>
</td>
  </tr>
</table>